# Note:
This is really my first attempt to learn Time Series. I'm sure that, despite the fact that the I'm really tried on EDA to get some data characteristics (and I'm happy about the EDA), The data formatting and model training is not what I'm proud of. I keep this notebook as a score record for the next iterations to come.

You can find the other version with the highter score here:

* [2# attempt : Focusing on data formatting and expanding windows](https://www.kaggle.com/code/thanakr/beginner-s-project-2nd-attempt)
* [3# attempt : Pipeline, Feature Selection, More complicated models](https://www.kaggle.com/thanakr/beginner-s-project-3-pipeline-and-models)

# Intro

Here is my very first attempt to learn the time series data exploration and machine learning. After studied many MOOCS and some articles, I think the only way to make me understand is to practice the knowledge with some project. And here I am. This is still a work in progress. I don't think I can do everything in one go, thus the contents will be added eventually. If someone wander around and see this notebook, feel free to leave any comments or suggestions in the discussion panel (I'm pretty sure that, for the first project, my code is dirty). I would be really appreciated with any comment for further improvment.

**Note:** Since this is a competition used in kaggle's time series course and the score I get just form copying the example that used statsmodels.tsa.deterministic.CalenderFourier() and statsmodels.tsa.deterministic.DeterministicProcess is **0.51090**. Let's see what score I'd get from doing the task again without the structured guide.

**Reference:**

https://www.kaggle.com/ilyakondrusevich/54-stores-54-models

https://www.kaggle.com/hiro5299834/store-sales-ridge-voting-bagging-et-bagging-rf  

https://www.kaggle.com/andrej0marinchenko/hyperparamaters#DeterministicProcess  

https://www.kaggle.com/ekrembayar/store-sales-ts-forecasting-a-comprehensive-guide  

# The Plan

1. define the goal
2. define problem
2. get aquainted the source dataset
    * data exploration
3. reserch models to use
4. data processing + feature engineering
5. built model(s)
6. validate model(s)
7. test the model(s)
8. submit the prediction to evauate the project by the submission score (dont forget to log the sore and changes in each iteration)
9. summarize the insight gained from the process
10. What could have done better??
11. Notes



# 1. define the goal

This is the kaggle's eduacationnal dataset for learning time series prediction. The task is to predict the sales of (thoundsands) of pruducts of an Equadorian shopping mall.

To do that, Kaggle have provided few datasets to be used as a features in the ML trainig dataset.

The data consist of 7 datasets namely:
1. **training.csv** : the training dataset
2. **transaction.csv** : log of the amount of daily 1transaction across all the stores
3. **stores.csv** : every store informations such as area etc.
4. **holiday_events.csv** : list of holiday held in Equador 
5. **oil.csv** : Historical price of crude oil (Equador main economic drive is the oil production)
6. **test.csv** : dataset for testing the model
7. **sample_submission.csv** : example of submission csv format

All the data will be further explored in the data exploration part.

Note: the task also state some additional informations that might be handy later on.
1. Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
2. A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.


First of all let's take a note about the goal of the competition, which is:
    
### Predict the price of products in the next 15 days from the lastday of the training data.

Let's take a look at test.csv and sample_submission.csv

In [ ]:
from warnings import simplefilter
simplefilter("ignore")  # ignore warnings to clean up output cells

import gc # for garbage cleaning

%config Completer.use_jedi = False # for autocomplete 

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
oil_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
holiday_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
store_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
train_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transaction_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
sample = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')

In [ ]:
test_data.head()

In [ ]:
sample.head()

We need to predict the sales on 2017-08-16 to 2017-08-31 for various kind of product and submit with this format.

# 2. Define the problem

Since this is the time series prediction problem, I will utilize the linear regression models to predict the expected sales for each product. However, there are so many features and complimentary datasets. Therefore, additional model might be useful too.


# 3. The data exploration

Here, I planed to explore these areas within all the datasets. In order to get the idea of what might be an interesting features to train the model.

Task:

1. **Total sales** : Hope to see an overall trend or the some spike in sales (perhaps the earthquake would have a big effect.)
2. **Daily sales by each stores** : Hope to see the trend and pattern in sales throughout the timeframe.
3. **Sales by product family, by time** : Hope to see the trend and pattern in sales throughout the timeframe.
4. **Store Location/Cluster Effect on sales** : This might be a good feature in further sale prediction.
5. **Onpromotion effect on sales**
6. **Cycle and Seasonal Effect on total sales** : Hope to see some bi-weekly effect or something interesting.
7. **Map the holiday to the sales record:** I guess that holiday will boost the sales.
8. **Calculate the correlation in change in oil price to change in total sales:** At this point I assume that the fluctuation in oil price won't affect the sales in short term. Since the change in oil price take times to effect the national income and ultimately business activites..
9. **Explore the transaction data** 

In [ ]:
train_data.head()
#seems like the store is close on new year day since they are all 0 sales in 1 Jan.

In [ ]:
train_data.info()
#over 3m. lines of data with a date formatted as a string dtype

In [ ]:
print('there are {} different product families'.format(train_data.family.nunique()))


In [ ]:
print('there are {} different stores'.format(train_data.store_nbr.nunique()))


train_data consist of several column and over 3 million rows
* id -> will be dropped
* date -> need to format to dt and use as index
* store number -> 1 to 54
* (product) family
* sales (trying to predict this)
* onpromotion. -> gives the total number of items in a product family that were being promoted at a store at a given date

In [ ]:
#let's format the datetime and use as index and drop the id column.
process_train = train_data.copy()

del train_data

process_train['date'] = pd.to_datetime(process_train['date'])
process_train = process_train.set_index('date')
process_train = process_train.drop('id',axis = 1)
process_train[['store_nbr','family']].astype('category') #to reduce ram usage
process_train

let's check the possible missing data / observation.

In [ ]:
process_train.isna().sum()

check the missing date (missing the entire observation).

Let's begin by checking the time-lag between each observation.

In [ ]:
def count_day_gap(df):
    temp = df.reset_index().groupby(['date']).sales.sum()
    return (temp.index[1:]-temp.index[:-1]).value_counts()

In [ ]:
count_day_gap(process_train)

4 rows of the data are missing as we see that the 2 days time-gap are presented 4 time. Where are thay?

In [ ]:
temp = process_train.reset_index().groupby(['date']).sales.sum().to_frame()
gap = (temp.index[1:]-temp.index[:-1]).to_list()
gap.insert(0,'first day') #add to gap list to have the same length as temp to combine it together
temp['gap'] = gap
#print(temp.gap.unique())

day_skip = temp.groupby('gap').get_group(temp.gap.unique()[2])
day_skip

In [ ]:
del temp

2013 - 2014 - 2015 -2016 Christmas days (20xx-12-25) are all missing. 

Let's see if there are other days that the total sales across country is zero. 

In [ ]:
process_train.groupby('date').sales.sum().sort_values().head(10)

Besides Christmas day, there are no other day that have zero sales. The Christmas will be add back in the holiday EDA section.


### Let's see a total sales combined

First of all, let's see total sales as a stackpolot. I could get the idea of what's selling well and the major trend of total sales. Also, I will mark the earthquake to see if the quake noticably affect the sales.

In [ ]:
date_fam_sale = process_train.groupby(['date','family']).sum().sales
unstack = date_fam_sale.unstack()
unstack = unstack.resample('1M').sum()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.set(title="'Total Monthly Sales From each Family")
plt.stackplot(unstack.index,unstack.T,labels=unstack.T.index)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.axvline(x=pd.Timestamp('2016-04-16'),color='black',linestyle='--',linewidth=4,alpha=0.5)
plt.text(pd.Timestamp('2016-04-20'),30000000,'  The Earthquake',rotation=360,c='black',size=16)
plt.xticks(rotation=90)
plt.legend(loc='lower center',bbox_to_anchor=(0.5,-0.2),ncol=11)
plt.show()

In [ ]:
del unstack
del date_fam_sale

**Interpretation:** The stackplot shows that only few product families accounted for a majority of sales. And the earthquake may cause a spike in sales for a short while. The steep drop at the end of the data is caused by the 2017-08 data that has been given from 2017-08-1 to 2017-08-15. So the drop is assosiated with the missing of the other half month.

The color is still confusing. Let's do the barchart to rank the product which are sold the most.

In [ ]:
month_family = process_train.groupby('family').resample('M').sales.sum() #resample to monthly sales 

fig, ax = plt.subplots(figsize=(7,7))
plt.barh(month_family.groupby('family').sum().sort_values().index,month_family.groupby('family').sum().sort_values())
ax.set(title='Total Sales by Product Family')
plt.show()


let's calculte the proportaion of sales by category


In [ ]:
total_sale = month_family.sum()
family_sale = month_family.groupby('family').sum().sort_values()
proportion = ((family_sale/total_sale)*100).sort_values(ascending=False)
proportion = pd.DataFrame(proportion)
proportion.head()

In [ ]:
del family_sale

The product families which accounted for the most sales are
1. GROCERY I - 31.99%
2. BEVERAGES - 20.21%
3. PRODUCE - 11.43%
4. CLEANING - 9.08%
5. DAIRY - 6%

Just for the top five, they have accounted for more than 75% of total sales already.

Let's take a look at daily total sales.

In [ ]:
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(18, 7))
ax.set(title="'Total Sales Across All Stores")
total_sales = process_train.sales.groupby("date").sum()
plt.plot(total_sales)

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.xticks(rotation=70)
plt.axvline(x=pd.Timestamp('2016-04-16'),color='r',linestyle='--',linewidth=4,alpha=0.3)
plt.text(pd.Timestamp('2016-04-20'),1400000,'The Earthquake',rotation=360,c='r')


plt.show()

**Interpretation:** Overall, I see an uptrend in total sales across the country since 2013. Let's check why there are those super deep valleys near many year's end. My guess now is that all the shop are closed at the New Year or New Year Eve for holiday so there is no sales. I will reconfirm this again at the holiday analyses.

Noted that the earthquake marked with the red dotted band seems to boost the sales few days after.

### Let's plot each store sale by date

In [ ]:
#create a dic that contain each store total daily sale.
daily_sale_dict = {}
for i in process_train.store_nbr.unique():
    daily_sale = process_train[process_train['store_nbr']==i]
    daily_sale_dict[i] = daily_sale

In [ ]:
fig = plt.figure(figsize=(30,30))
for i in daily_sale_dict.keys():
    plt.subplot(8,7,i)
    plt.title('Store {} sale'.format(i))
    plt.tight_layout(pad=5)
    sale = daily_sale_dict[i].sales
    sale.plot()
    plt.axvline(x=pd.Timestamp('2016-04-16'),color='r',linestyle='--',linewidth=2,alpha=0.3) #mark the earthquake

**Interpretation:** There are so many "gaps" or "white space" in some stores sales record. I guess they are from either temporary store close or the store not yet operate. 

One thing to note is that, from times to times, there will be a sharp spike in sales (ex. store # 35 which had 3 spikes around the year's end of 2014, 2015, 2016. This may need futher scrutinization.

Considering the earthquake, it affected the sales variedly from store to store. Some store such as 18,20,21,etc.. saw a great one time spike. Some store such as store 5,26,35,etc.. didn't have any huge sales changes. However, for store 53, it's the gamechanger. Store 53 had gone through a long duration of sale increase for years.

The main theme here is that, on average, store sales are in uptrend since 2013.

### What about the sales by Product Family.

In [ ]:
by_fam_dic = {}
fam_list = process_train.family.unique()

for fam in fam_list:
    by_fam_dic[fam] = process_train[process_train['family']==fam].sales

In [ ]:
fig = plt.figure(figsize=(30,50))

for i,fam in enumerate(by_fam_dic.keys()):
    plt.subplot(11,3,i+1)
    plt.title('{} sale'.format(fam))
    plt.tight_layout(pad=5)
    sale = by_fam_dic[fam]
    sale.plot()
    plt.axvline(x=pd.Timestamp('2016-04-16'),color='r',linestyle='--',linewidth=2,alpha=0.3) #mark the earthquake


**Interpretation:** Each family has their own selling paterns. However, Frozen Food and School and Office Supplies shown highly seasonal cycle.
* Frozen Food : Sell more on New Year
* School and Office Supplies: Sale more around AUG

In [ ]:
del by_fam_dic
del fam_list

### What about the stores themselves.

The competition data description explicitly describe that "cluster is a grouping of similar store". This might explan some of the different sales patterns in the subplots above. Let's see what can I get from this.

In [ ]:
store_data.head(3)

In [ ]:
#add the cluster to the process_train df
join_df = process_train.merge(store_data,on='store_nbr')

#get the missing time after merge back in place
join_df.set_index(process_train.index)
join_df['date'] = process_train.index
join_df = join_df.set_index('date')

join_df.head()

I'm curious about the store_data Let's compare the average sales between types and alo between cluster. Hopefully I can see some pattern. 

In [ ]:
 def show_type_df(join_store_type_df):
    mean_sales_type = join_store_type_df.groupby('type').sales.mean()
    median_sales_type = join_store_type_df.groupby('type').sales.median()
    number=join_store_type_df.groupby('type').store_nbr.nunique()

    type_df = pd.DataFrame((mean_sales_type,median_sales_type,number))
    type_df = type_df.T
    type_df.columns = ['mean','median','number of store']

    return type_df

In [ ]:
show_type_df(join_df)

Although type C and D are most common store type, but the biggest individual performance of the sales came from type A and D. In fact, type C is the worst proforming store in term of sales.

### What about the store cluster.

In [ ]:
def show_cluster_summary(join_store_type_df):
    mean_sales_cluster = join_store_type_df.groupby('cluster').sales.mean()
    median_sales_cluster = join_store_type_df.groupby('cluster').sales.median()
    number=join_store_type_df.groupby('cluster').store_nbr.nunique()

    cluster_df = pd.DataFrame((mean_sales_cluster,median_sales_cluster,number))
    cluster_df = cluster_df.T
    cluster_df.columns = ['mean','median','number of store']

    return cluster_df.sort_values('mean', ascending=False)

In [ ]:
show_cluster_summary(join_df)

Cluster #5 is unique. It has only 1 store but contribute to the biggest individual cluster sales. The other's are as the dataframe states. I'm not sure if cluster will be a reall good featues since the sales are not really distictive except the #5. We reconfirm this thought see in the model later.

### let's see the City characteristic



In [ ]:
def show_city_df(join_store_type_df):    
    mean_sales_city = join_store_type_df.groupby('city').sales.mean()
    median_sales_city = join_store_type_df.groupby('city').sales.median()
    number=join_store_type_df.groupby('city').store_nbr.nunique()

    city_df = pd.DataFrame((mean_sales_city,median_sales_city,number))
    city_df = city_df.T
    city_df.columns = ['mean','median','number of store']

    return city_df.sort_values('mean', ascending=False)

In [ ]:
show_city_df(join_df)

In [ ]:
del join_df

This looks really interesting. Quito seem to be the main operating city for our chain store and accounted for the most product sales across the country.

Last thing I'd like to explore within stores is a heatmap coorelation between the sales pattern of each stores.

In [ ]:
import seaborn as sns

a = process_train[["store_nbr", "sales"]] #slice only the two columns
a["ind"] = 1 # create a column of ones

#cumulate the 1 column til the last observation. Ultimately, count the lines for a store.
a["ind"] = a.groupby("store_nbr").ind.cumsum().values 

#create a corelation pivot table between sales of each stores
a = pd.pivot(a, index = "ind", columns = "store_nbr", values = "sales").corr() 

mask = np.triu(a.corr()) # slice the top triangle of the a.corr() dataFrame

plt.figure(figsize=(20, 20))
sns.heatmap(a,
         annot=True,
         fmt='.1f',
         square=True,
         mask=mask,
         linewidths=1,
         cbar=False)
plt.title("Correlations among stores",fontsize = 20)
plt.show()

#ref: https://www.kaggle.com/ekrembayar/store-sales-ts-forecasting-a-comprehensive-guide/notebook

In [ ]:
del a
del mask

In [ ]:
gc.collect()

All the store sales seems to go together except the store number 52. 

But.... scrolling up to the grid of store sales above (just ctrl+f "Let's plot each store sale by date"). 
Store 52 is just a newly opened store. and has almost no sales record compared to the others. 

In fact, all the dark cell contain lower corelation from store 20,21,22,42,52 are stores that are newly operated compared to the others.

let's check the first entry of store 20 to make sure.

In [ ]:
store_20 = process_train.groupby(['store_nbr','date']).sales.sum().loc[20]
store_20.loc['2013-01-01':'2015-01-01']

In [ ]:
del store_20

really no sales at all

### Effect of promotion

the competition overview state that the **onpromotion** column "gives the total number of items in a product family that were being promoted at a store at a given date."

let's take a look with the corelation between onpromotion and sales.

In [ ]:
print('Spearman Rank Correlation = {}'.format(
    process_train.sales.corr(process_train.onpromotion,method='spearman')))

There are  coorelation between promotion and sales of certain family of product.

The reason I choose Spearman rank correlation here is because the default (Pearson's) correlation need the distribution of both variable to be normal distributed. While the promotion itself is define by the marketing plan.

[For more detail](https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/correlation-pearson-kendall-spearman/)

### Let's explopre trend, seasonal, and cycles

Just for simplicity of the first attempt, I'm not gonna use any determininstic process or Fourier series to determine the cycle of the sales. Instead, I will used rolling window average to project a trend and used as a feature for trend.

As for cycle, I don't think I can overlook this feature since the day of the week (or month of year) are crucial in retail business. I will use one-hot encoding for and feed as a feature.

First, let's make sure that the seasonal effect exist.

In [ ]:
from datetime import datetime as dt

def show_dow_sales():
    day_group = process_train.reset_index()[['date','sales']]
    day_group = day_group.groupby('date')
    day_group = day_group.sales.mean().to_frame()
    day_group['dow'] = day_group.index.day_of_week
    day_group = day_group.groupby('dow').sum()
    plt.bar(day_group.index,day_group['sales'])
    plt.title('Average sales on day of week')
    plt.show()

In [ ]:
def show_month_group_sale():
    month_group = process_train['sales'].to_frame()
    month_group['moy'] = month_group.index.month
    month_group = month_group.groupby('moy').sales.mean().to_frame()
    plt.bar(month_group.index,month_group['sales'])
    plt.title('Average sales on Month of Year')
    plt.show()

In [ ]:
show_dow_sales()
show_month_group_sale()

On average, business is good in the weekends and quarter 4.

After extracting some basic seasonality, let's explore the effect of holiday.

### Holiday Effect on sales

* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday). 

In [ ]:
holiday_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",index_col='date',parse_dates=['date'])
holiday_data

Let's check if Quito (the main operating area) is here 

In [ ]:
holiday_data.locale_name.value_counts().head()

Quito is here and it has the most regional holidays. 

Didn't see New Year and Christmas day in the holiday_data. This day is so impactful since the sales across the counrty is dropped to about zero. Let's add it back.

In [ ]:
ny_dic = {'type': 'Holiday','locale':'National','locale_name':'Ecuador','description': 'New Year Day','transferred':'False'}
ny_date = pd.to_datetime(['2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01','2018-01-01'])

cm_dic = {'type': 'Holiday','locale':'National','locale_name':'Ecuador','description': 'Christmas Day','transferred':'False'}
cm_date = pd.to_datetime(['2012-12-25','2013-12-25','2014-12-25','2015-12-25','2016-12-25','2017-12-25','2018-12-25'])

In [ ]:
for date in ny_date:
    holiday_data.loc[date] = ['Holiday','National', 'Ecuador', 'New Year day','False']
    
for date in cm_date:
    holiday_data.loc[date] = ['Holiday','National', 'Ecuador', 'Christmas day','False']

In [ ]:
holiday_data = holiday_data.sort_index()

From taking a look at the holiday df above, I think there are someting need formatting.
* Group Football Events(They might have the different effect from other ceremonial event.
* Deal with transfered events
* Emphasize Cyber Monday and Black Friday (Since these are practically shopping day.)
* Creat a dummy DF that contain the 
    * date 
    * type of event (one-hot)
        * is_football
        * is_Earthquake
        * is_shopping_event
        * other events
    * where the event was held (one-hot)
        * Other dummy columns for the location
    * work day or not


(My guide for formatting holiday's here :https://www.kaggle.com/andrej0marinchenko/hyperparamaters)

In [ ]:
calendar = pd.DataFrame(index = pd.date_range('2013-01-01','2017-08-31'))
calendar = calendar.join(holiday_data).fillna(0)
del holiday_data
calendar

In [ ]:
calendar['dow'] = calendar.index.dayofweek+1
calendar['workday'] = True
calendar.loc[calendar['dow']>5 , 'workday'] = False #make work_day false for sat and sun (6/7 in dow)

calendar.head()

Next, I'm setting workday in each **National** Holiday to ba False no matter dayofweek. Local holiday maybe a normal working day for the nation.

In [ ]:
calendar.loc[(calendar['type']=='Holiday') & (calendar['locale'].str.contains('National')), 'workday'] = False
calendar.loc[(calendar['type']=='Additional') & (calendar['locale'].str.contains('National')), 'workday'] = False
calendar.loc[(calendar['type']=='Bridge') & (calendar['locale'].str.contains('National')), 'workday'] = False
calendar.loc[(calendar['type']=='Transfer') & (calendar['locale'].str.contains('National')), 'workday'] = False

#some holiday are explicitly said it is workday
calendar.loc[calendar['type']=='Work Day' , 'workday'] = True

Let's check the transfered holiday. They supposed to become working day if they are not Sat and Sun. Let's see that.

In [ ]:
calendar.where(calendar['transferred'] == True).dropna()

They are all weekday (since 'dow' are all less than 5). Therefore, format the work_day to True

In [ ]:
calendar.loc[(calendar['transferred'] == True), 'workday'] = True

Format the football to be the one same kind of event. Do the same for the earthquake that last multiple days.

In [ ]:
calendar.where(calendar['description'].str.contains('futbol')).dropna()

In [ ]:
calendar['is_football'] = 0
calendar['is_eq'] = 0

calendar.loc[(calendar['is_football'] == 0) & (calendar['description'].str.contains('futbol')), 'is_football'] = 1
calendar.loc[(calendar['is_eq'] == 0) & (calendar['description'].str.contains('Terremoto')), 'is_eq'] = 1

In [ ]:
calendar.where(calendar['is_football']==1).dropna().head() # just checking

In [ ]:
calendar.where(calendar['is_eq']==1).dropna().head() # just checking

let's test if the event (such as football / earthquake / Black Friday) affect sales.

In [ ]:
calendar.loc[calendar['is_football']==1,'description'] = 'football'
calendar.loc[calendar['is_eq']==1,'description'] = 'earthquake'

#for simplicity just format the description of the football event and earthquake to be the same

In [ ]:
sales = process_train.groupby('date').sales.sum()
event = calendar[calendar['type']=='Event']

event_merge = event.merge(sales,how='left',left_index=True,right_index=True)
event_merge

del sales
del event

In [ ]:
print('mean of daily sale across country: {}'.format(process_train.groupby('date').sales.sum().mean()))
print('--------------------')

print(('mean of sale across country in event day: {}'.format(event_merge.groupby('description').sales.mean())))

comparing those means, black friday and Dia de la Madre seems to not effect total sales much while all the other event affect the sales more.

Now, from the plan

* Group Football Events(They might have the different effect from other ceremonial event.
* Deal with transfered events
* Emphasize Cyber Monday and Black Friday (Since these are practically shopping day.)
* Creat a dummy DF that contain the
    * date
    * type of event (one-hot)
    * is football
    * is Earthquake
    * is cyber monday
    * is black friday
    * are other events
    * where the event was held (one-hot)
    * is workday
    * day of week
    
let's do the dummy DF beginnig by drop the transfered holidays.



In [ ]:
calendar.head()

In [ ]:
calendar['workday'] = calendar['workday'].map({False:0,True:1})
calendar['transferred'] = calendar['transferred'].map({'False':0,False:0,True:1})

In [ ]:
calendar['is_ny'] = 0
calendar['is_christmas'] = 0
calendar['is_shopping'] = 0

calendar.loc[calendar['description'] == 'New Year day', 'is_ny'] = 1
calendar.loc[calendar['description'] == 'Christmas day', 'is_christmas'] = 1
calendar.loc[calendar['description'] == 'Black Friday', 'is_shopping'] = 1
calendar.loc[calendar['description'] == 'Cyber Monday' , 'is_shopping'] = 1

In [ ]:
calendar.loc['2014-12-25'].to_frame().T #check the christmas

In [ ]:
calendar = calendar.drop(['type','locale'], axis=1)

In [ ]:
calendar.head()

In [ ]:
locale_dummy = pd.get_dummies(calendar['locale_name'],prefix='holiday_')
calendar = locale_dummy.join(calendar,how='left')
calendar = calendar.drop('locale_name',axis=1)

del locale_dummy

In [ ]:
calendar_checkpoint = calendar

In [ ]:
calendar_checkpoint = calendar_checkpoint.drop('description',axis = 1) #don't need description anymore we have dummied them all
calendar_checkpoint = calendar_checkpoint[~calendar_checkpoint.index.duplicated(keep='first')] 
calendar_checkpoint = calendar_checkpoint.iloc[:,1:-1]

In [ ]:
calendar_checkpoint

In [ ]:
del calendar
gc.collect()

Now the dummy table for event are ready to be join to the main sales datafame. Noted that I leave the day of week in ordinal manner since the sales seem to increase toward the weekend. Please let me know if I souldn't do this.

### The effect of oil price on the sales

I put an assumption that the short term effect from change in oil price to the sales is negligible. Let's see if my guessy assumption holds. 

In [ ]:
oil_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")

In [ ]:
oil_data.head(4)

I need to refill the NaN data and also maybe add a price change / percent chance to be additional features.

let's check the missing date. (remember that the training data are from 2013-01-01 to 2017-08-15)

In [ ]:
pd.date_range(start = '2013-01-01', end = '2017-08-15' ).difference(oil_data.index)

# this useful code is from https://stackoverflow.com/questions/52044348/check-for-any-missing-dates-in-the-index

So much missing let's interpolate the missing data.

In [ ]:
oil_data['date'] = pd.to_datetime(oil_data['date'])
oil_data = oil_data.set_index('date')

In [ ]:
oil_data = oil_data.resample('1D').sum()

In [ ]:
oil_data.reset_index()

In [ ]:
pd.date_range(start = '2013-01-01', end = '2017-08-15' ).difference(oil_data.index)

No missing date now. Let's interpolate

In [ ]:
oil_data['dcoilwtico'] = np.where(oil_data['dcoilwtico']==0, np.nan, oil_data['dcoilwtico'])
oil_data['interpolated_price'] = oil_data.dcoilwtico.interpolate()

In [ ]:
oil_data = oil_data.drop('dcoilwtico',axis=1)

In [ ]:
oil_data.head()

In [ ]:
oil_data['price_chg'] = oil_data.interpolated_price - oil_data.interpolated_price.shift(1)
oil_data['pct_chg'] = oil_data['price_chg']/oil_data.interpolated_price.shift(-1)

In [ ]:
oil_data.head()

In [ ]:
fig,ax = plt.subplots(figsize=(15,5))
plt.plot(oil_data['interpolated_price'])
plt.title('Oil Price over time')

plt.show()

Now let's calculate the correlation between oil price and total sales.
Let's begin by a scatterplot

In [ ]:
daily_total_sales = total_sales.copy()

In [ ]:
daily_total_sales.head()

In [ ]:
daily_total_sales = daily_total_sales.resample('1D').sum()
daily_total_sales

In [ ]:
oil_data.interpolated_price.loc['2013-01-01':'2017-08-15']

In [ ]:
plt.scatter(daily_total_sales,oil_data.interpolated_price.loc['2013-01-01':'2017-08-15'],alpha=0.2)
plt.ylabel('oil price')
plt.xlabel('daily total sales')
plt.show()

Cosidering the scatterplot there seems to be no obvious linear correlation between both of them. the most I could say is that higher the oil price, lower the sales. However let's consider another angle. The oil price changes vs total sales changes. We already have oil price change and percentage change. Let's prepare the daily_total_sales.

In [ ]:
daily_total_sales = pd.DataFrame(daily_total_sales)
daily_total_sales['sales_chg'] = daily_total_sales['sales']-daily_total_sales['sales'].shift(1)
daily_total_sales['sales_pct_chg'] = daily_total_sales['sales_chg']/daily_total_sales['sales'].shift(-1)

daily_total_sales.head()

Let's find correlation betwenn the amount change and percentage change of both oil and sales.

In [ ]:
print('Spearman Rank Correlation = {}'.format(
    process_train.sales.corr(process_train.onpromotion,method='spearman')))

In [ ]:
print('Pearson Correlation between oil price change and total sales change = {}'.format(
oil_data.price_chg.corr(daily_total_sales.sales_chg,method='pearson')))
print('Spearman Rank Correlation between oil price change and total sales change = {}'.format(
oil_data.price_chg.corr(daily_total_sales.sales_chg,method='spearman')))

print('============================================================================')

print('Pearson Correlation between % oil price change and % total sales change = {}'.format(
oil_data.pct_chg.corr(daily_total_sales.sales_pct_chg,method='pearson')))
print('Spearman Rank Correlation between oil price change and total sales change = {}'.format(
oil_data.pct_chg.corr(daily_total_sales.sales_pct_chg,method='spearman')))

### What about Oil Lag??
Let's check another angle. The coorelation between the oil price lag and the sales just in case that the effect from change in oil price talk times to be fully accounted into people shopping pattern.

In [ ]:
import  statsmodels.graphics.tsaplots # for partial autocorrelation

ax = statsmodels.graphics.tsaplots.plot_pacf(oil_data.interpolated_price.dropna(), 
                                                 lags=31,
                                                 title = 'oil lag PACF')

Oil price lag 10,15,26,27 is good to used. Let's see if there are any corelation to the sales.

In [ ]:
oil_lag = [10,15,26,27]
for lag in oil_lag:
    oil_data['price_lag_{}'.format(lag)] = oil_data.interpolated_price.shift(lag)

In [ ]:
oil_for_lag_coor = oil_data.dropna()
oil_for_lag_coor.head()

In [ ]:
oil_for_lag_coor = oil_for_lag_coor.merge(daily_total_sales,how='inner',left_index=True,right_index=True)

In [ ]:
oil_for_lag_coor = oil_for_lag_coor[['price_lag_10','price_lag_15','price_lag_26','price_lag_27','sales']]
oil_for_lag_coor.head()

In [ ]:
fig, ax = plt.subplots()
lag_col =  ['price_lag_10','price_lag_15','price_lag_26','price_lag_27']

for lag in lag_col:
    plt.scatter(oil_for_lag_coor['sales'],oil_for_lag_coor[lag],alpha=0.25)
    plt.title('Oil Lag {} vs Sales'.format(lag))
    plt.xlabel('oil price lag {}'.format(lag))
    plt.ylabel('amount sold')
    plt.legend()
    plt.show()

Noting Obvoius again. Higher the lag price, lower the sales. This just mightbe the result from the fact that the oil price had been dropping while the sales is getting better. A correlation, maybe, a causation, I don't know. 

PS. If someone could shed some light on this topic, please elaborate on the discussion panel please.

As of now, I woudn't use the oil price as a feature.

In [ ]:
del [oil_lag,oil_data,oil_for_lag_coor,lag_col]

gc.collect()

### The Transaction record

In [ ]:
transactions = transaction_data.copy()
transactions = transactions.set_index('date')

del transaction_data

I really feel like this must be a very powerful feature. The transaction amount must be highly corelated with the sales. Let's process it by
1. Make a dic cotain store_nbr as a key and transaction as elements
2. Interpolate the missing date in the dic
3. scatterplot the transaction with total sales record
4. decide if assumption is good to be used in the model 

In [ ]:
transactions.index = pd.to_datetime(transactions.index)

In [ ]:
transactions.head()

In [ ]:
def transaction_sales_dic(transaction_df,sale_dic):
    transaction_dic = {}
    sale_dict = sale_dic.copy()

    for i in transaction_df['store_nbr'].unique():
        store_transacion = transaction_df.loc[transaction_df['store_nbr'] == i]
        transaction_dic[i] = store_transacion['transactions']
        
    for i in sale_dict.keys():
        sale_dict[i] = sale_dict[i].groupby(['date','store_nbr']).sales.sum()
        sale_dict[i] = sale_dict[i].reset_index()
        sale_dict[i] = sale_dict[i].drop('store_nbr', axis=1)
        sale_dict[i] = sale_dict[i].groupby('date').sales.sum()
            
    return transaction_dic, sale_dict
        
def  series_merge_inner_index(dic1, dic2):
    merged_dic = {}
    for key in dic1.keys():
        merged_dic[key] = dic1[key].to_frame().merge(dic2[key].to_frame(), how='inner',
                                                    left_index=True, right_index=True)
    return merged_dic

In [ ]:
transaction_dic, sale_dic = transaction_sales_dic(transactions,daily_sale_dict)
merged_sales_transaction = series_merge_inner_index(transaction_dic, sale_dic)
merged_sales_transaction[1] #just for checking the outcome

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for key in merged_sales_transaction.keys():
    plt.scatter(merged_sales_transaction[key].transactions,
                merged_sales_transaction[key].sales)

plt.title('Transaction vs Amount Sold')
plt.xlabel('transactions number')
plt.ylabel('amount sold')
plt.show()

This is very obvious that transaction number and amount sold are highly corelated. This feature will be used in the model. Considering the nature of test dataset that didn't provide the amount of transaction on those days (otherwise there will be a data leakage), Transaction must be used as a lag feature. In fact, it need to be a 16 or more lag since the test horizon is 15 days from the last day of the training dataset.

Let's see transaction lag. Beginning with the transaction lag partial correlation for picking the good lags to use.

In [ ]:
import  statsmodels.graphics.tsaplots # for partial autocorrelation

for store_nbr in merged_sales_transaction.keys():
    ax = statsmodels.graphics.tsaplots.plot_pacf(merged_sales_transaction[store_nbr].transactions, 
                                                 lags=range(16,31),
                                                 title = 'Store {} transactions lag'.format(store_nbr))

Interestingly, Transaction lag of 21,22,28 seem to be good used for almost all stores. will used them as feature.

### From what I plan to explore:

1. Total sales
2. Daily sales by each stores
3. Sales by product family, by time
4. Store Location/Cluster Effect on sales
5. Onpromotion effect on sales
6. Cycle and Seasonal Effect on total sales
7. Map the holiday to the sales record
8. Calculate the correlation in change in oil price to change in total sales
9. Explore the transaction data




### This is what I have
1. The overall trend is up for total sales 
2. the top 5
   * GROCERY I - 31.99%
   * BEVERAGES - 20.21%
   * PRODUCE - 11.43%
   * CLEANING - 9.08%
   * DAIRY - 6% 
   
   accounted for more than 75% of total sales
   * Frozen Food / School and Office Supply, though not a major share, has a highly cyclic selling pattern
  
3. Some store have a lot of missing dates. I guess it's not operate or not yet open on those days
   * Closing days besides, all the other store have high corelation in daily sales change.
4. There are some interesting paterns within the sales need further explorations such as:
    * The earthquale effect
    * The New Year
5. Store characteristic are categorized by cluster, format. Let's see their significance later when we build and train the model.
    * Type and Cluster seem to have some effect on sales.
    * The sores located most in Pichincha and Guayas
    
6. There are strong corelation between onpromotion and sales of the product on promotion
7. On average, sales are higher around the end of the week  on weekly basis an higher around Q4 on yearly basis
8. The dummy DF for holiday is ready to be merge to the main sales DF
9. Oil price seem to have no corralation to the sales amount.
10. Transaction number is highly corelated to total sales.
    * Will be used as lag feature (lag 21,22,28)

# 4. Model research: Which model should I use here.

Since this is the first attempt and I already have a benchmark score from the kaggele time series course of **0.51090**, I think I will begin with simple linear regression. Later, I plan to refine or replace the model or methodology used for building the model for better outcome.

# 5. Data processing and Feature engineering

With the exploration sofar. I plan to train the model with these features
* Daily Sales lag (Will check again for different lag correlations)
* Total daily sales lag Across the country
* The amount of the promotion for that product family
* Store Area (to be crossed with the locale name of holiday)
* Features from Holiday
    * type
    * locale name
    * day of week
    * is_worlday
    * Earthquake
    * Football
    * New Year
    * shopping events (such as Black Friday)
* Daily Transaction Lag for each shop

We are already have the holiday_df ready to be merge. Let's begin making inputs dataframe. 


### Lag Feature
First thing first, let's explore the sales lag correlation to the sales itself.

Also the test dataset come in different product family and different store, I don't want to construct the lag for each product family and each store and the majority of the product sold are consist of only 5 family. Therefore,I'm gonna use only the lag from store total sales.


In [ ]:
import  statsmodels.graphics.tsaplots # for partial autocorrelation


daily_store_sale_dict = {}
for i in daily_sale_dict.keys():
    daily_store_sale_dict[i] = daily_sale_dict[i].groupby(['date','store_nbr']).sales.sum().to_frame()    

In [ ]:
del daily_sale_dict

In [ ]:
daily_store_sale_dict[1].head() #just checking for store 1 dic

In [ ]:
for i in daily_store_sale_dict.keys():
    daily_store_sale_dict[i] = daily_store_sale_dict[i].droplevel(1) 
    #remove store_nbt from index leaving only date

In [ ]:
for i in daily_store_sale_dict.keys():

    ax = statsmodels.graphics.tsaplots.plot_pacf(daily_store_sale_dict[i],lags=14, 
                                                 title = 'store {} PA'.format(i))

I thin I will keep the 1 2 3 4 5 6 7 8 13 14 Lags.

The lag is prepared. Let' start constructing the input df along the way.

Let's combine the process_train and test for the ease of formatting

In [ ]:
process_train

In [ ]:
test_data.head()

In [ ]:
test_data['sales'] = 0
test_data = test_data[['id','date','store_nbr','family','sales','onpromotion']]

test_data

In [ ]:
process_train['id'] = process_train.reset_index().index

In [ ]:
process_train = process_train.reset_index()
process_train = process_train[['id','date','store_nbr','family','sales','onpromotion']]
process_train

In [ ]:
merged_train = pd.concat([process_train,test_data])
merged_train = merged_train.set_index(['date','store_nbr','family'])
merged_train

In [ ]:
merged_train

In [ ]:
del test_data
gc.collect()

let's add the store location as another index.

In [ ]:
store_location = store_data.drop(['state','type','cluster'],axis=1) # the city column is the same scale as holiday location.
store_location = store_location.set_index('store_nbr')
store_location = pd.get_dummies(store_location,prefix='store_loc_')
store_location

merge the store location into inputs DF 

In [ ]:
inputs = merged_train.reset_index().merge(store_location,how='outer',left_on='store_nbr',right_on=store_location.index)

In [ ]:
inputs

In [ ]:
del store_location
del merged_train
gc.collect()

Next, lag features: sales lag, total daily sales lag, transaction lag. I have prepared the store sales lag and tranasation lag, let's do the total country sales lag.

In [ ]:
total_sales

Need to project the index further to 2017-08-31.

In [ ]:
total_sales_to_scale = pd.DataFrame(index=pd.date_range(start='2013-01-01',end='2017-08-31'))
total_sales_to_scale = total_sales_to_scale.merge(total_sales,how='left',left_index=True,right_index=True)
total_sales_to_scale = total_sales_to_scale.rename(columns={'sales':'national_sales'})

In [ ]:
total_sales_to_scale

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mmScale = MinMaxScaler()
mmScale.fit(total_sales_to_scale['national_sales'].to_numpy().reshape(-1,1))

total_sales_to_scale['scaled_nat_sales'] = mmScale.transform(total_sales_to_scale['national_sales'].to_numpy().reshape(-1,1))


In [ ]:
total_sales_to_scale

Next, we need to make some lags to the scaled_nat_sales. The lag will also be 16-30 for the same reason with other lag.

In [ ]:
import  statsmodels.graphics.tsaplots # for partial autocorrelation

ax = statsmodels.graphics.tsaplots.plot_pacf(total_sales_to_scale['scaled_nat_sales'].dropna(), 
                                                 lags=range(16,31),
                                                 title = 'PACF of national sales')

lag 16 - 24 and 27,28 seems good to be used here.

In [ ]:
lags= [16,17,18,19,20,21,22,23,24,27,28]
for lag in lags:
    total_sales_to_scale['nat_scaled_sales_lag{}'.format(lag)] = total_sales_to_scale['scaled_nat_sales'].shift(lag)

In [ ]:
total_sales_to_scale = total_sales_to_scale.drop(['national_sales','scaled_nat_sales'],axis=1) 
#don't think the current sales will be practical in real situation

In [ ]:
total_sales_to_scale.reset_index().tail() #reset index for ease of merge

In [ ]:
inputs = inputs.merge(total_sales_to_scale.reset_index(),how='left',left_on='date',right_on='index')

In [ ]:
del total_sales_to_scale

In [ ]:
inputs.columns #just checking what are in the df

In [ ]:
inputs.drop(['index'],axis=1,inplace=True) #column named 'index' in dt format don't need it anymore

Now, let's continue on individual store sales lag. From the lag ananyses above, I have conclude that the lags which will be used here are lag = [1 2 3 4 5 6 7 8 13 14]. Let's put it to good use.

In [ ]:
inputs

In [ ]:
lags = [1, 2, 3, 4, 5, 6, 7, 8, 13, 14]
for lag in lags:
    inputs['store_fam_sales_lag_{}'.format(lag)] = inputs['sales'].shift(lag)

In [ ]:
inputs.columns

Now add the transaction lag to the inputs.

In [ ]:
transactions

still left the row from 2017-08-16 to 2017-08-31 and notice that someday, some store won't open (such as new year that we only have store 25. How do I map this to the inputs dataframe?? 

In [ ]:
store_nbr = range(1,55)
dates = pd.date_range('2013-01-01','2017-08-31')
mul_index = pd.MultiIndex.from_product([dates,store_nbr],names=['date','store_nbr'])
df = pd.DataFrame(index=mul_index)

In [ ]:
df.reset_index()

In [ ]:
transactions.reset_index()

In [ ]:
df_transaction = df.reset_index().merge(transactions.reset_index(),
                                        how='left',
                                        left_on=['date','store_nbr'],
                                        right_on=['date','store_nbr']
                                       )

In [ ]:
df_transaction.fillna(0, inplace=True)

In [ ]:
df_transaction.loc[30020:30026] #just checking

In [ ]:
#df_transaction.rename(columns={'level_0':'date','level_1':'store_nbr'}, inplace = True)

In [ ]:
df_transaction

Now adding the lag. Remember the meaningful lag for transaction are lag = [21,22,28]

In [ ]:
lags = [21,22,28]
for lag in lags:
    df_transaction['trans_lag_{}'.format(lag)] = df_transaction['transactions'].shift(lag)

In [ ]:
df_transaction = df_transaction.drop('transactions',axis=1) #in real life daily transaction came after day's end

In [ ]:
df_transaction = df_transaction.fillna(0)

In [ ]:
df_transaction.loc[30030:30040] #ust checking

Next, merge the df_transaction to the inputs df.

In [ ]:
inputs

In [ ]:
inputs = inputs.merge(df_transaction, how='left', left_on = ['date','store_nbr'],right_on = ['date','store_nbr'])

In [ ]:
inputs

Next, add the holiday features into the inputs DF

In [ ]:
calendar_checkpoint.reset_index().tail()

In [ ]:
inputs = inputs.merge(calendar_checkpoint,how='left',left_on=['date'],right_on=calendar_checkpoint.index)

In [ ]:
inputs.columns

let's check for missing data.

In [ ]:
pd.set_option('display.max_rows',None)
inputs.isna().sum()

In [ ]:
pd.reset_option('display.max_rows','display.max_columns')

Drop all the NaN (which are all lagging features)

In [ ]:
inputs.dropna(inplace = True)
inputs.isna().sum().sum()

In [ ]:
inputs = inputs.set_index('date')

In [ ]:
inputs.tail()

From all the features list I planned to use.
* Sales lag ---- DONE
* Total daily sales lag Across the country ---- DONE
* The amount of the promotion for that product family ---- DONE
* Store Area (to be crossed with the locale name of holiday) ---- DONE
* Features from Holiday ---- DONE
    * type ---- DONE
    * locale name ---- DONE
    * day of week ---- DONE
    * is_worlday ---- DONE
    * Earthquake ---- DONE
    * Football ---- DONE
    * New Year ---- DONE
    * shopping events (such as Black Friday) ---- DONE
* Daily Transaction Lag for each shop ---- DONE

Let's make the training data and testing data.

In [ ]:
y_train = inputs.loc['2013-01-01':'2017-08-15', 'sales']
y_train.tail()

In [ ]:
x_train = inputs.loc['2013-01-01':'2017-08-15'].drop(['sales','id'],axis=1)
x_train = x_train.reset_index()
x_train = x_train.set_index(['date','store_nbr','family'])


In [ ]:
x_train.tail()

In [ ]:
x_train.describe()

In [ ]:
x_test = inputs.loc['2017-08-16': ]
test_id = x_test['id'] #Keep for later

x_test.drop(['sales','id'],axis = 1,inplace = True)

x_test = x_test.reset_index()
x_test = x_test.set_index(['date','store_nbr','family'])
x_test

# 6. Built model(s) and Validate model(s)

Finally it's time for the model training. As I mentioned above, this very first attempt will be for the simple linear regression model and the metrics will be the "Root Mean Squared Logarithmic Error" (RMSLE), in other words, the submission score itself. I am aimimg to surpass the benchmark score of **0.5109** from the Kaggle time series course. I didn't expect anythin more than "abysmal" for the result in this first attempt with simple model and inexperience feature eugineering. However, I hope that the next iterations to come will yield better results. 

### Let's do it.

In [ ]:
from sklearn.linear_model import LinearRegression

ln = LinearRegression()
ln.fit(x_train,y_train)

In [ ]:
y_pred = ln.predict(x_test)

# 7. Test the models and Submission

Since this is a regression prediction, I think I won't need a validation data. I will just measure the model by the score.

In [ ]:
len(y_pred)

prepare the submission.

First, make sure that the id are matched between the train data and y_pred.

In [ ]:
sample

In [ ]:
test_id 

The id are matched. Let's submit

In [ ]:
sample['sales'] = y_pred
sample

In [ ]:
# The actual submission
sample.to_csv('submission.csv', index = False) # Submit

# 8. Problems Notes

These are the notes I plan to use in the next iteration of this project or others. I plan to do this project as a freely unguide - simple project to see what would happen if I want to try time series with some crude skills and knowledge. I hope that, as I keep going, I will develop better understanding and more mastery. I hope that this note book in the next next iteration will be better and more robust.


### Problems
1. Need more coding, statistic and ML mastery to fully tackle the problem 
    * -> Practicing
2. Still Have no idea how to deal with the different selling patterns between different Product Families.
    * -> Some great notebooks are dealing with this by using more complicated models. I will try to understand them.
3. I feel like the Oil price must be beneficial for training the model. Just don't know how to utilize it.
    * -> Explore more idea from other notebooks
4. Need more practice using Pandas snd Matplotlib.
5. Sometimes I'm not sure how to impliment the insight from the EDA into the model.
6. Lacking of project planning experience seems to cost me a lot of time fixing and refixing the code and also the procedure. For example, I had construct the holiday dummy table just to find that I need to to the entire formatting and dummying again in the main calendar DF since the Holiday DF are note date complete.
    * -> I guess this is how the lesson is learned
7. Memory leak: I was writing code without thinking about the memory leak. This is really bad practice and the first time I got it in ML. This stall my entire process and need to be fix right away.
    * -> Concern more 
    * -> I Google and found this great [notebook](https://www.kaggle.com/pavansanagapati/14-simple-tips-to-save-ram-memory-for-1-gb-dataset/comments). Upvoted.
        * simpler methods I think I'm gonna use here in this notebook are:
            * Free Memory using gc.collect()
            * Datatype Conversions¶ (for convert dummy or categorical data into category using data.astype('category')
            * Dealing with memory leak by using in-function variable (which will be cleared after the function done executing.
    * When merging BIG dataframes, it might be better for limited memory to convert the bigger one in to csv at a managable chunk using chunksize= pharameter.
    * Still, due to my lake of experience the DF structure in have in mind still won't work. It got over 4M rows. Need to fix immediately.


# 9. Conclusion

The result is really bad. Let's breakdown the model to see why.

Calculate the Coeficients of the model.

In [ ]:
coef = pd.DataFrame(data = ln.coef_, index=x_train.columns,columns=['coef'])
coef['abs_value'] = abs(coef['coef'])

coef = coef.sort_values(by='abs_value',ascending=False)


pd.set_option('display.max_rows',None)
display(coef)
pd.reset_option('display.max_rows')


This is very peculiar. I'm not expected that the top features will be those of the store locations (with exactly equal weight), also the weight is really high. Something is really really wrong here

Next' the intercept.

In [ ]:
ln.intercept_

In intercept is really strange. begining at a super big number. This is a sigh of really big residuals.

Let's plot the predictions. I might find something even worse.

In [ ]:
result = x_test
result['predicted_sales'] = y_pred
result = result['predicted_sales'].groupby(['date','store_nbr']).sum()

result

In [ ]:
result = result.unstack()

pd.set_option('display.max_columns',None)
result

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))
plt.plot(result)
plt.title('Predicted sales for each store')
plt.show()

Too bad. each store follow the same pattern since I use only one model to predict each store. Few (store 10 16 33 36) has high peak during the last days. I wonder what do thay have in common? let's dig deeper

# 10. What could have done better? + Idea for the next iteration.

This is what I have in mind after the first run. I would be very appreciated to all suggestion in the discussion panel. 

1. I forgot to include the insight that some store don't sell some product. Those combination sales could be set to zero flat.
2. Removing the store locations from the inputs might be a good idea. (Perhaps Check the colinearity between all the features)
3. Perhaps, each store should have the model trained individually since different store has different sales pattern, therefore, diffenet weight characteristics.
4. Take in account only the holiday from the main operating area (Quito City) and national holiday
5. Instead of dropna for many lag feature, maybe I should fill_na with zero to preserve the observation.
6. Try scale the onpromotion too
7. Dummy the DOW variable.
8. Try using MA instead of lag feature
9. Using Pipeline in the data processing
10. Using Sliding windows technique for training.

# 11. Footnote

It's been a really daunting task for a first-timer like me just to do this from the beginning to the submission. For this simple-educational competition, and using only the linear regression model, It took me many more days than I expected. However, after the submission, I felt really happy that I had finally finised the first attempt.

The score I got is **3.44178**, which, is noting less than an "abysmal" just like the level I expected to have compared to the **0.5xx** from the Kaggle Course. Still, I'm very happy about the process and what I've learn from practicing with this notebook.

The code here, I'm sure, must be very scrappy, but for the next iteration with more experience and more complicated model (I'm thinking about DeterministicProcess(), Fourier Combination, Forest etc.), I hope to produce better result and better understanding of data science.

For Kaggle and everyone that read through my notebook, thank you.